In [1]:
import tensorflow as tf
from numpy import prod, savez, load
from pprint import pprint
import sys, os

In [8]:
class FC_Network(object):
    """A neural network of fully connected layers.
    
    First layer has Leaky_ReLU activation with a trainable alpha. Other layers have ReLU activation.
    Input sample has dropout at rate 1 - sample_keep_prob.
    activations have dropout at rate 1 - fcl_keep_prob.
    """
    
    def __init__(self,
                 name,
                 layer_nodes,
                 dtype = tf.float32,
                 num_freq_channels = 1024,
                 learning_rate = 0.0001,
                 cost_name = 'MSE',
                 threshold = 0.00625,
                 g_shift = 0.01,
                 log_dir = 'logs/'):
        
        self.log_dir = log_dir
        
        self.params = {"num_freq_channels" : num_freq_channels,
                       "learning_rate" : learning_rate,
                       "layer_nodes" :layer_nodes,
                       "cost_name" : cost_name,
                       "threshold" : threshold,
                       "g_shift" : g_shift,
                       "dtype" : dtype,
                       "name" : name}

        
    def load_params(self, path):
        """Load in the parameters of an old network, but keep the current network name."""
    
        sys.stdout.write('\rLoading Netowrk Parameters')
        a = load(path + '.npz')
        d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
        
        name = self.params['name']
        self.params = d['data1arr_0'][()]
        
        # make sure to set the name for the current network 
        self.params['name'] = name
        
        
    def print_params(self):
        """Prints netwwork parameters"""
        pprint(self.params)
        
    def _save_params(self):
        param_direc = self.log_dir + self.params['name'] + '/params/'
        """Not safe - will overwrite existing file."""
        if not os.path.exists(param_direc):
            os.makedirs(param_direc)
        savez(param_direc + self.__class__.__name__, self.params)   



        

        
    def create_graph(self):
        sys.stdout.write('\rCreating Network Graph')
        self._save_params()
        
        self.layers = []
        self.layer_names = ['layer_{}'.format(i) for i in range(len(self.params['layer_nodes']))]
        
        tf.reset_default_graph()
        
        with tf.variable_scope('keep_probs'):

            self.sample_keep_prob = tf.placeholder(self.params['dtype'], name = 'sample_keep_prob')

            self.fcl_keep_prob = tf.placeholder(self.params['dtype'], name = 'fcl_keep_prob')  
            
        with tf.variable_scope('sample'):

            self.X = tf.placeholder(self.params['dtype'], shape = [None, 1024], name = 'X')
            self.X = tf.nn.dropout(self.X, self.sample_keep_prob)
            
        with tf.variable_scope('input_layer'):
            
            b = tf.get_variable(name = 'biases', shape = [self.params['layer_nodes'][0]],
                                initializer = tf.contrib.layers.xavier_initializer())
            
            w = tf.get_variable(name = 'weights', shape  = [1024, self.params['layer_nodes'][0]],
                                initializer = tf.contrib.layers.xavier_initializer())
            
            self.leaky_relu_alpha = tf.get_variable(shape = [],
                                                    name = 'leaky_relu_alpha',
                                                    dtype = self.dtype,
                                                    constraint = lambda x: tf.clip_by_value(x, 0, 0.3),
                                                    initializer = tf.zeros_initializer())
            
            layer = tf.nn.leaky_relu(tf.matmul(self.X, w) + b, alpha = self.leaky_relu_alpha)
            layer = tf.nn.dropout(layer, self.fcl_keep_prob)
            self.layers.append(layer)
            
        for i in range(len(self.params['layer_nodes'])):
            if i > 0:
                with tf.variable_scope('layer_%d' %(i)):
                    layer = tf.contrib.layers.fully_connected(self.layers[i-1], self.params['layer_nodes'][i])
                    if self.params['layer_nodes'][i] >= 256:
                        layer = tf.nn.dropout(layer, self.fcl_keep_prob)
                    self.layers.append(layer)

                    
        with tf.variable_scope('prediction'):
            self.predictions = tf.contrib.layers.fully_connected(self.layers[-1], 1)


        with tf.variable_scope('targets'):
            self.targets  = tf.placeholder(self.params['dtype'], shape = (None, 1), name = 'targets')

            
            
        with tf.variable_scope('costs'):

            error = tf.subtract(self.targets, self.predictions, name = 'error')
            squared_error = tf.square(error, name = 'squared_difference')

            with tf.variable_scope('mean_inverse_shifted_gaussian'):
                with tf.variable_scope('normal_distribution'):
                    
                    sigma = tf.constant(self.params['threshold'], name = 'sigma')
                    normal_dist = tf.contrib.distributions.Normal(0.0, sigma, name = 'normal_dist')
                    gaussian_prob = normal_dist.prob(error, name = 'gaussian_prob')
                    shift = tf.constant(self.params['g_shift'], name = 'shift')
                    shifted_gaussian = tf.add(gaussian_prob, shift, name = 'shifted_gaussian')        

                self.MISG = tf.reduce_mean(tf.divide(1.0, shifted_gaussian), name = 'mean_inverse_shifted_gaussian')

            with tf.variable_scope('mean_squared_error'):
                self.MSE = tf.reduce_mean(squared_error)
        
        with tf.variable_scope('train'):
            cost = self.MSE if self.params['cost_name'] == 'MSE' else self.MISG
            LR = tf.constant(self.params['learning_rate'], name = 'learning_rate')
            self.optimizer = tf.train.AdamOptimizer(LR, epsilon=1e-08).minimize(cost)
            
        with tf.variable_scope('logging'):  

            with tf.variable_scope('image'):
                self.image_buf = tf.placeholder(tf.string, shape=[])
                epoch_image = tf.expand_dims(tf.image.decode_png(self.image_buf, channels=4), 0)

            with tf.variable_scope('percent_within_threshold'):
                self.PWT = 100.*tf.reduce_mean(tf.cast(tf.less_equal(tf.abs(self.targets - self.predictions), sigma), self.params['dtype']) )

            tf.summary.histogram(name = 'targets', values = self.targets)
            tf.summary.histogram(name = 'predictions',values =  self.predictions)
            tf.summary.scalar(name = 'leaky_relu_alpha', tensor = self.leaky_relu_alpha)
            tf.summary.scalar(name = 'MSE', tensor = self.MSE)
            tf.summary.scalar(name = 'MISG', tensor = self.MISG)
            tf.summary.scalar(name = 'PWT', tensor = self.PWT)
            tf.summary.image('prediction_vs_actual', epoch_image)
            self.summary = tf.summary.merge_all()